In [110]:
import pandas as pd
import numpy as np

In [111]:
data = pd.read_excel("questionaire_raw.xlsx")
df = pd.DataFrame(data)

In [112]:
# Translate and remane columns manually
df = df.rename(columns={
        "你有大細腳的問題嗎？": "suffering_from_unequal_foot_size", 
        "你的性別": "Gender",
        "哪隻腳比較大？": "bigger_foot",
        "左右腳大約相差的碼數是？": "feet_size_difference_EU",
        "你平時買鞋的大約碼數？": "usual_shoe_size_EU",
        "參考的鞋款/牌子？(如converse/nike，可選填）": "reference_shoe_brand",
    })

In [113]:
# Tidy up column values
df["suffering_from_unequal_foot_size"] = df["suffering_from_unequal_foot_size"].replace("有", True).replace("沒有", False)
df["Gender"] = df["Gender"].replace("男", "Male").replace("女", "Female")
df["bigger_foot"] = df["bigger_foot"].replace("左", "Left").replace("右", "Right")
df["feet_size_difference_EU"] = df["feet_size_difference_EU"].replace("小於半個碼", 0.25).replace("半個碼", 0.5).replace("一個碼", 1).replace("1.5個", 1.5)

/var/folders/ds/sqj592l55_d1lx7mf8hn7w1w0000gn/T/ipykernel_66695/3856691696.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["suffering_from_unequal_foot_size"] = df["suffering_from_unequal_foot_size"].replace("有", True).replace("沒有", False)
/var/folders/ds/sqj592l55_d1lx7mf8hn7w1w0000gn/T/ipykernel_66695/3856691696.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["feet_size_difference_EU"] = df["feet_size_difference_EU"].replace("小於半個碼", 0.25).replace("半個碼", 0.5).replace("一個碼", 1).replace("1.5個", 1.5)


In [114]:
df

,suffering_from_unequal_foot_size,Gender,bigger_foot,feet_size_difference_EU,usual_shoe_size_EU,reference_shoe_brand
0,True,Male,Right,0.50,43.5,Adidas
1,True,Male,Right,0.25,43.0,Converse
2,False,Male,NaN,NaN,NaN,NaN
3,False,Male,NaN,NaN,NaN,NaN
4,False,Female,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
895,True,Female,Left,0.25,40.0,NaN
896,False,Female,NaN,NaN,NaN,NaN
897,True,Male,Right,0.50,44.0,NaN
898,False,Male,NaN,NaN,NaN,NaN


In [115]:
# Add two extra columns to df: left_foot_size_estimated_estimated and right_foot_size_estimated
df["left_foot_size_estimated"] = np.nan
df["right_foot_size_estimated"] = np.nan

In [116]:
# If the interviewee has unequal foot size, calculate the left and right foot size
for index, row in df.iterrows():
    if row["suffering_from_unequal_foot_size"] and row["feet_size_difference_EU"] >= 0.5:
        if row["bigger_foot"] == "Left":
            df.at[index, "left_foot_size_estimated"] = row["usual_shoe_size_EU"]
            df.at[index, "right_foot_size_estimated"] = row["usual_shoe_size_EU"] - row["feet_size_difference_EU"]
        else:
            df.at[index, "left_foot_size_estimated"] = row["usual_shoe_size_EU"] - row["feet_size_difference_EU"]
            df.at[index, "right_foot_size_estimated"] = row["usual_shoe_size_EU"]

In [117]:
df.to_excel("questionaire_tidied_up.xlsx", index=False)

In [118]:
# translate the column names with Translator library, deprecate because of inaccuracy

# from translate import Translator

# translator= Translator(from_lang="zh", to_lang="en")
# for i in range(len(df.columns)):
#     column_in_eng = translator.translate(df.columns[i])
#     print(column_in_eng)